In [1]:
import os

import evaluate
import numpy as np
from datasets import load_dataset
from dotenv import load_dotenv
from transformers import TrainingArguments, Trainer, \
    DataCollatorWithPadding, AutoModelForSequenceClassification, AutoTokenizer

from peft import LoraConfig, TaskType, get_peft_model

In [2]:
load_dotenv()

True

In [3]:
model_id = 'roberta-large'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.getenv('HUGGINGFACE_TOKEN'))
model = AutoModelForSequenceClassification.from_pretrained(model_id,
                                                           trust_remote_code=False,
                                                           num_labels=2,
                                                           token=os.getenv('HUGGINGFACE_TOKEN'))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Use PEFT for effeciency.

In [5]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    bias='none',
    lora_dropout=0.1,
)

In [6]:
model = get_peft_model(model, lora_config)

In [7]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): M

### EDA

In [8]:
imdb = load_dataset("imdb")
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [9]:
def preprocess_function(examples):
    # Tokenize the reviews
    text_tokenized = tokenizer(examples['text'], padding='max_length', truncation=True)
    return text_tokenized


tokenized_train = imdb['train'].map(preprocess_function, batched=True)
tokenized_test = imdb['test'].map(preprocess_function, batched=True)
tokenized_unsupervised = imdb['unsupervised'].map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [10]:
tokenized_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [11]:
tokenized_test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [12]:
def compute_metrics(eval_pred):
    load_accuracy = evaluate.load("accuracy")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    return {"accuracy": accuracy, }

In [13]:
training_args = TrainingArguments(
    output_dir="./saved_model/training",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    warmup_steps=1000,
    do_train=True,
    do_eval=True,
    bf16=True,
)

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\aksid\PycharmProjects\ReviewCategoriser\venv\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [16]:
%%time

training_result = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.167500,0.147959,0.960560
2,0.170600,0.137451,0.963320
3,0.152500,0.141213,0.963600


CPU times: total: 16min 2s
Wall time: 54min 14s


In [17]:
# compute train results
metrics = training_result.metrics
max_train_samples = len(tokenized_train)
metrics["train_samples"] = min(max_train_samples, len(tokenized_train))

In [18]:
# compute evaluation results
metrics = trainer.evaluate()
max_val_samples = len(tokenized_test)
metrics["eval_samples"] = min(max_val_samples, len(tokenized_test))

In [19]:
metrics

{'eval_loss': 0.13745084404945374,
 'eval_accuracy': 0.96332,
 'eval_runtime': 297.6179,
 'eval_samples_per_second': 84.0,
 'eval_steps_per_second': 10.5,
 'epoch': 3.0,
 'eval_samples': 25000}

### Performance Metrics

| eval_loss | eval_accuracy | eval_runtime | eval_samples_per_second | eval_steps_per_second | epoc | eval_samples |
|---|---|---|---|---|---|---|
| 0.13478049635887146 | 0.96352 | 294.8682 | 84.784 | 10.598 | 3.0 | 25000 |

### Test!

In [20]:
import random

N = len(tokenized_unsupervised)

idx = random.randint(1, N)

example = tokenized_unsupervised[idx]
text = example['text']
text

'My town is a small one and we can only have the chance to see the big Hollywood productions.I had to wait for months to see this movie and I found it at Blockbusters yesterday.The cast is a dream cast for me I always liked Peter Mullan after My name is Joe.Milla Jovovich is an actress which I really find passionate.When the film was over I could not believe how bad it was.Meaningless details,bad picture quality and unbelievably bad story. I am so sorry for the cast and the director.'

In [21]:
tokenised = tokenizer(text, return_tensors='pt')
model = model.to('cpu')
res = model(**tokenised)

In [22]:
import torch.nn.functional as F

probabilities = F.softmax(res.logits, dim=1)
predicted_class = probabilities.argmax(dim=1)
predicted_class[0]

tensor(0)

In [23]:
trainer.save_model(f'peft/{model_id}')

In [24]:
model.num_parameters()

357199876